## Avaliação de Veículos
Criação de um modelo de Machine Learning para avaliação de veículos com base em características, após,
será gerado um modelo, salvo em disco, para ser utilizado através de uma API, feito em Flask.

In [1]:
# Importando as bibliotecas
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Carregando e analisando os dados

In [2]:
df = pd.read_excel('car.xlsx')

In [3]:
df.head()

,buying,maint,doors,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


**buying: preço de compra  
maint: preço da manutenção  
doors: quantidade de portas  
persons: quantidade de pessoas no veículo  
lug_boot: tamanho do bagageiro  
safety: segurança estimada do veículo  
class: avaliação final do veículo**

In [4]:
# Retirando informações
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   buying    1728 non-null   object
 1   maint     1728 non-null   object
 2   doors     1728 non-null   object
 3   persons   1728 non-null   object
 4   lug_boot  1728 non-null   object
 5   safety    1728 non-null   object
 6   class     1728 non-null   object
dtypes: object(7)
memory usage: 94.6+ KB


In [5]:
# Verificando nulos
df.isnull().sum()

buying      0
maint       0
doors       0
persons     0
lug_boot    0
safety      0
class       0
dtype: int64

In [6]:
# Alterando dados categóricos para numéricos
buying = {'low':1, 'med':2, 'high':3, 'vhigh':4}
maint = {'low':1, 'med':2, 'high':3, 'vhigh':4}
doors = {'5more':5}
persons = {'more':5}
lug_boot = {'small':1, 'med':2, 'big':3}
safety = {'low':1, 'med':2, 'high':3}

# Armazenando em outro dataset
df_replace = df.replace({'buying':buying, 'maint':maint, 'doors':doors,
                         'persons':persons, 'lug_boot':lug_boot, 'safety':safety})

In [7]:
df_replace.head()

,buying,maint,doors,persons,lug_boot,safety,class
0,4,4,2,2,1,1,unacc
1,4,4,2,2,1,2,unacc
2,4,4,2,2,1,3,unacc
3,4,4,2,2,2,1,unacc
4,4,4,2,2,2,2,unacc


In [8]:
# Verificando as informações do novo dataset
df_replace.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   buying    1728 non-null   int64 
 1   maint     1728 non-null   int64 
 2   doors     1728 non-null   int64 
 3   persons   1728 non-null   int64 
 4   lug_boot  1728 non-null   int64 
 5   safety    1728 non-null   int64 
 6   class     1728 non-null   object
dtypes: int64(6), object(1)
memory usage: 94.6+ KB


In [9]:
# Dividindo inputs e classes
x = df_replace.drop('class', axis=1)
y = df_replace['class']

In [10]:
# Importdando bibliotecas para treinamento
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Execução dos treinamentos

### Treinamento do KNN com GRidSearch

In [11]:
knn = KNeighborsClassifier()
valor = list(range(1,31))
valores = {'n_neighbors' : valor}
gs_knn = GridSearchCV(knn, valores, cv=5, scoring='accuracy')

In [12]:
gs_knn.fit(x, y)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                         13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                         23, 24, 25, 26, 27, 28, 29, 30]},
             scoring='accuracy')

In [13]:
# Melhores resultados
print(f'Melhor resultado: {gs_knn.best_score_}')
print(f'Melhores parâmetros: {gs_knn.best_params_}')

Melhor resultado: 0.8507598224009383
Melhores parâmetros: {'n_neighbors': 21}


### Treinamento do SVM com Gridsearch

In [14]:
sup = SVC()
lista_C = [1, 5, 10, 15, 20, 25, 30]
lista_gamma = [1, 5, 10, 15, 20, 25, 30]
valores = {'C':lista_C, 'gamma':lista_gamma}
gs_svm = GridSearchCV(sup, valores, cv=5, scoring='accuracy')

In [15]:
gs_svm.fit(x,y)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [1, 5, 10, 15, 20, 25, 30],
                         'gamma': [1, 5, 10, 15, 20, 25, 30]},
             scoring='accuracy')

In [16]:
# Melhores resultados
print(f'Melhor resultado: {gs_svm.best_score_}')
print(f'Melhores parâmetros: {gs_svm.best_params_}')

Melhor resultado: 0.8397486805730082
Melhores parâmetros: {'C': 5, 'gamma': 1}


### Treinamento do Random Forestcom Gidsearch

In [17]:
random = RandomForestClassifier()
lista = [70, 80, 90, 100, 110, 120, 130]
valor_random = {'n_estimators':lista}
gs_random = GridSearchCV(random, valor_random, cv=5, scoring='accuracy')

In [18]:
gs_random.fit(x,y)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'n_estimators': [70, 80, 90, 100, 110, 120, 130]},
             scoring='accuracy')

In [19]:
# Melhores resultados
print(f'Melhor resultado: {gs_random.best_score_}')
print(f'Melhores parâmetros: {gs_random.best_params_}')

Melhor resultado: 0.8223741308536482
Melhores parâmetros: {'n_estimators': 110}


# Efetuando a validação do melhor resultado

In [20]:
# Separando dado de treino e teste
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, random_state=42)

In [21]:
# Aplicando Validação Cruzada
print(pd.crosstab(y_teste, gs_knn.predict(x_teste), margins=True))

col_0  acc  good  unacc  vgood  All
class                              
acc     91     0     12      0  103
good     4    11      0      0   15
unacc    2     0    292      0  294
vgood    2     1      0     17   20
All     99    12    304     17  432


In [22]:
# Aplicando métricas de classifição
print(metrics.classification_report(y_teste, gs_knn.predict(x_teste)))

              precision    recall  f1-score   support

         acc       0.92      0.88      0.90       103
        good       0.92      0.73      0.81        15
       unacc       0.96      0.99      0.98       294
       vgood       1.00      0.85      0.92        20

    accuracy                           0.95       432
   macro avg       0.95      0.87      0.90       432
weighted avg       0.95      0.95      0.95       432



# Gerando e armazenando modelo em disco

In [23]:
import joblib

In [24]:
joblib.dump(gs_knn, 'model.joblib')

['model.joblib']